# Exercise 3 : Text Classification (Logistic Regression, Naive Bayes, K-NN)

Classify reviews of Musical Instruments present in Amazon using the following algorithms: <br>
1) Logistic Regression <br>
2) Naive Bayes <br>
3) K- Nearest Neighbour <br>
(Note: Reviews with overall score <=4 is to be treated as negative and those with overall score >4 is to be treated as positive)

The dataset can be obtained from http://jmcauley.ucsd.edu/data/amazon/ http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering R. He, J. McAuley WWW, 2016

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from pylab import *
import nltk
import warnings
warnings.filterwarnings('ignore')

In [ ]:
review_data = pd.read_json('data_ch3/reviews_Musical_Instruments_5.json', lines=True)
review_data.head()

In [ ]:
review_data['overall'].value_counts()

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
review_data['cleaned_review_text'] = review_data['reviewText'].apply(\
lambda x : ' '.join([lemmatizer.lemmatize(word.lower()) \
    for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x)))]))

In [ ]:
review_data[['cleaned_review_text', 'reviewText', 'overall']].head()

In [ ]:
tfidf_model = TfidfVectorizer(max_features=500)
tfidf_df = pd.DataFrame(tfidf_model.fit_transform(review_data['cleaned_review_text']).todense())
tfidf_df.columns = sorted(tfidf_model.vocabulary_)
tfidf_df.head()

In [ ]:
#Let's consider review with overall score <= 4 to be negative (encode it as 0) 
#and overall score > 4 to be positive (encode it as 1)

review_data['target'] = review_data['overall'].apply(lambda x : 0 if x<=4 else 1)
review_data['target'].value_counts()

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(tfidf_df,review_data['target'])
predicted_labels = logreg.predict(tfidf_df)

In [ ]:
logreg.predict_proba(tfidf_df)[:,1]

In [ ]:
review_data['predicted_labels'] = predicted_labels

In [ ]:
pd.crosstab(review_data['target'], review_data['predicted_labels'])

# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(tfidf_df,review_data['target'])
predicted_labels = nb.predict(tfidf_df)

In [ ]:
nb.predict_proba(tfidf_df)[:,1]

In [ ]:
review_data['predicted_labels_nb'] = predicted_labels

In [ ]:
pd.crosstab(review_data['target'], review_data['predicted_labels_nb'])

# K-Nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(tfidf_df,review_data['target'])
review_data['predicted_labels_knn'] = knn.predict(tfidf_df)

In [ ]:
pd.crosstab(review_data['target'], review_data['predicted_labels_knn'])

Reference / Citation for the dataset: http://jmcauley.ucsd.edu/data/amazon/
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz    
Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering
R. He, J. McAuley WWW, 2016